In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

# Problem Setup

In [ ]:
from trajectory import Trajectory
from environment import Environment
from global_variables import DIM

n_anchors = 3 #number of anchors
n_positions = 20 #number of robot sample positions
n_complexity = 4 #model complexity

trajectory = Trajectory(n_complexity, dim=DIM, model='polynomial')
environment = Environment(n_anchors)
basis = trajectory.get_basis(n_samples=n_positions)

environment.set_random_anchors(seed=1)
trajectory.set_coeffs(seed=1)

print(basis.shape)
sample_points = trajectory.get_sampling_points(basis=basis, seed=1)

plt.figure()
environment.plot()
trajectory.plot(basis, color='orange')
#plt.axis('off')
#plt.savefig('traj_setup.png')

In [ ]:
from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse

n_it = 3
D_noiseless = get_D_topright(environment.anchors, sample_points)
print(D_noiseless.shape)
basis = trajectory.get_basis(times=times)

for i in range(n_it):
    mask = create_mask(n_positions, n_anchors, strategy='minimal', dim=DIM, n_complexity=n_complexity, seed=i+1)
    
    try:
        D_missing = np.multiply(D_noiseless, mask)
        X = alternativePseudoInverse(D_missing, environment.anchors, basis)
        X_noiseless = alternativePseudoInverse(D_noiseless, environment.anchors, basis)
    
        assert np.allclose(X_noiseless, trajectory.coeffs)
        assert np.allclose(X, trajectory.coeffs, atol=1e-3)
        print('Result exact for seed', i)
        
    except AssertionError:
        print('diff noiseless:', X_noiseless - trajectory.coeffs)
        print('diff noisy:    ', X - trajectory.coeffs)
        print('Result not exact for seed', i)
        
    except Exception as e:
        print('Singular matrix for seed {}? Error message:'.format(i)) 
        print(e)

In [ ]:
from constraints import get_C_constraints
import scipy

C = trajectory.coeffs
C_prime = np.r_[C.flatten(), (C.T.dot(C)).flatten()]


T_A, T_B, b = get_C_constraints(D_missing, environment.anchors, basis)
#T_A, T_B, b = get_C_constraints(D_noiseless, environment.anchors, basis)
T = np.c_[T_A, -T_B/2]

assert np.allclose(T.dot(C_prime), b)

print(T.shape, b.shape)
Lp, Up = scipy.linalg.lu(T, permute_l=True, )
P, L, U = scipy.linalg.lu(T, permute_l=False)

assert np.allclose(P.dot(L).dot(U), T)
assert np.allclose(P.dot(L), Lp)
assert np.allclose(U, Up)
print('L, U shapes:', L.shape, U.shape)

print(U.dot(C_prime))
for row_idx in range(U.shape[0]):
    num_nnz = trajectory.n_complexity * trajectory.dim
    nnz_elements = U[row_idx, :num_nnz].dot(C_prime[:num_nnz])
    z_elements = U[row_idx, num_nnz:].dot(C_prime[num_nnz:])
    print('should be zero: {:2.2f}'.format(z_elements), 'should be nonzero: {:2.2f}'.format(nnz_elements))

#P, L, U = scipy.linalg.lu(T.T, permute_l=False)

In [ ]:
fig = plt.matshow(L)
plt.gcf().set_size_inches(5, 5)
fig = plt.matshow(U)
plt.gcf().set_size_inches(7, 5)
fig = plt.matshow(trajectory.coeffs.reshape((-1, 1)))
plt.gcf().set_size_inches(5, 5)

#concat = np.c_[L, U]
#
#fig, ax = plt.subplots()
#fig.set_size_inches(10, 5)
#im = ax.matshow(concat)
#plt.colorbar(im)
#fig, axs = plt.subplots(1, 2)
#axs[0].matshow(Lp)
#axs[1].matshow(U)
#plt.show()

## SDP - based approach
### Noiseless case - single experiment

In [ ]:
print("Make sure that your cvxpy version is >= 1.0.6!")
import cvxpy
print("Your version:", cvxpy.__version__)

from solvers import OPTIONS, semidefRelaxationNoiseless

# We cane change the global variable OPTIONS here. 

#OPTIONS[cvxpy.SCS]["max_iters"] = 200
# Seems to have no effect: 
#OPTIONS[cvxpy.SCS]["use_indirect"] = False 
# Seems to have no effect either: 
#OPTIONS[cvxpy.SCS]["eps"] = 1e-1
# Seems to have no effect either: 
#OPTIONS[cvxpy.SCS]["scale"] = 1

# Fails completely without this:
#OPTIONS[cvxpy.CVXOPT]["kktsolver"] = "robust"

# have no effect:
#OPTIONS[cvxpy.CVXOPT]["feastol"] = 1e-3
#OPTIONS[cvxpy.CVXOPT]["reltol"] = 1e-5
#OPTIONS[cvxpy.CVXOPT]["abstol"] = 1e-5

# leads to faster non-convergence: 
#OPTIONS[cvxpy.CVXOPT]["refinement"] = 0

#OPTIONS[cvxpy.SCS]["verbose"] = False

basis = trajectory.get_basis(times=times)

X = semidefRelaxationNoiseless(D_noiseless, environment.anchors, basis, chosen_solver=cvxpy.CVXOPT)
print('should be identity:\n', X[:DIM, :DIM])
print('should be equal:\n', X[:DIM:, DIM:])
print(trajectory.coeffs)

In [ ]:
from solvers import alternativePseudoInverse
X = alternativePseudoInverse(D_noiseless, environment.anchors, basis)
print('should be equal:\n', X[:,:])
print(trajectory.coeffs)

### Noisy case - single experiment - NOT CURRENTLY WORKING

In [ ]:
from solvers import semidefRelaxation
from solvers import OPTIONS
#OPTIONS[cvxpy.SCS]["max_iters"] = 00

#X = semidefRelaxation(D_topright, environment.anchors, basis)
print('should be identity:\n', X[:DIM, :DIM])
print('should be equal:\n', X[DIM:, DIM:])
print(trajectory.coeffs)

## Investigate null space

In [ ]:
from constraints import *

ConstraintsMat, ConstraintsVec = get_constraints_matrix(D_noiseless, environment.anchors, basis)
ConstraintsMat=np.array(ConstraintsMat)
ConstraintsVec=np.array(ConstraintsVec)

print(np.isclose(ConstraintsMat@trajectory.Z_opt.flatten(),ConstraintsVec))

print(ConstraintsMat.shape)
print(ConstraintsVec.shape)
u, s, vh = np.linalg.svd(ConstraintsMat, full_matrices=True)
print(np.around(s,3))

In [ ]:
#construct right inverse and check meets constraints
num_zero_SVs = len(np.where(s<1e-10)[0])
Z_hat = vh[:-num_zero_SVs,:].T@np.diag(1/s[:-num_zero_SVs])@u[:,:len(s)-num_zero_SVs].T@ConstraintsVec #right inverse
Z_hat = Z_hat.reshape([DIM + n_complexity,DIM + n_complexity])
print(np.isclose(ConstraintsMat@Z_hat.flatten(),ConstraintsVec)) #should satisfy constraints since it's a right inverse
coeffs_hat = Z_hat[:DIM,DIM:]
print(np.isclose(trajectory.coeffs,coeffs_hat))

In [ ]:
print('find basis vectors of null space')
tmp = vh[-num_zero_SVs:,:]
print(tmp.shape)
nullSpace = []
for i in range(num_zero_SVs):
    nullSpace.append(tmp[i,:].reshape([DIM + n_complexity,DIM + n_complexity]))

nullSpace = np.array(nullSpace)
Z_hat2 = Z_hat+nullSpace[0,:]+2*nullSpace[1,:]+3*nullSpace[2,:]
print(np.isclose(ConstraintsMat@(Z_hat2.flatten()),ConstraintsVec))

In [ ]:
print(np.around(nullSpace[0,:],5))
print(np.around(nullSpace[1,:],5))
print(np.around(nullSpace[2,:],5))

### Solve as linear program

In [ ]:
# try linear program
from cvxopt import matrix, solvers

tmpMat=np.diag(s[:-num_zero_SVs])@vh[:-num_zero_SVs,:]
tmpVec = u[:,:len(s)-num_zero_SVs].T@ConstraintsVec
A = matrix(np.vstack([tmpMat,-tmpMat]))
b = matrix(np.hstack([tmpVec,-tmpVec]))
c = matrix(np.ones((DIM + n_complexity)*(DIM + n_complexity)))
sol=solvers.lp(c,A,b, solver='glpk') #doesn't work eith defualt solver but glpk good
print(sol)
Z_hat = np.array(sol['x']).reshape([DIM + n_complexity,DIM + n_complexity])
print(np.isclose(Z_hat,trajectory.Z_opt))

## MDS - based approach
### Noiseless case

In [ ]:
# find new coefficients
from baseline_solvers import customMDS
coeffs_est = customMDS(D_noiseless, basis, environment.anchors)
print(coeffs_est.shape)
print(coeffs_est)
print(trajectory.coeffs)

In [ ]:
from trajectory import Trajectory
trajectory_est = Trajectory(n_complexity, model='polynomial')
trajectory_est.set_coeffs(coeffs=coeffs_est)

plt.figure()
environment.plot()
trajectory_est.plot(basis=basis, color='red', linestyle=':')
trajectory.plot(basis=basis, color='green', linestyle='--')

### Noisy case

In [ ]:
from baseline_solvers import gradientDescent

sigma = 4

np.random.seed(1)
D_topright_noisy = D_noiseless + sigma * np.random.randn(*(D_noiseless.shape))
coeffs_est_noisy = customMDS(D_topright_noisy, basis, environment.anchors)

coeffs_est_noisy, costs = gradientDescent(
    environment.anchors, basis, coeffs_est_noisy, 
    D_topright_noisy,maxIters=50)
#print(checkStationaryPointSRLS(A,F,C_hat,DTR_tilde))
#plt.plot(costs)

In [ ]:
trajectory_est_noisy = Trajectory(n_complexity, model='polynomial')
trajectory_est_noisy.set_coeffs(coeffs=coeffs_est_noisy)

plt.figure()
environment.plot()
trajectory_est_noisy.plot(basis=basis, color='red', linestyle=':')
trajectory.plot(basis=basis, color='green', linestyle='--')